---
layout: post
title: Collectable Lesson Starter
description: Usage of compareTo and toString in Collectable framework
courses: { csa: {week: 15} }
type: ccc
---

## Abstract, Comparable, Implements, Interface

Establish class heirarchy, enforce definitions, and use compareTo.

In [38]:
/* This is parent class...
   Objective would be to push more functionality into parent Class 
   - Enforces consistent definition of child Class

 */
public abstract class Collectable implements Comparable <Collectable> {
	public final String masterType = "Collectable";
	private String type;	// extender should define their data type

	/* Enumerated interface of key types 
	 * an interface named KeyTypes is declared with a single method name(). 
	 * the Collectable class contains an abstract method getKey(), 
	 * which must be implemented by its subclasses. 
	 * must provide a method that returns an object implementing the KeyTypes interface.
	*/ 
	public interface KeyTypes {
		String name();
	}
	protected abstract KeyTypes getKey();  	// this method helps force usage of KeyTypes
	protected abstract KeyTypes getSortKey();  	// this method helps force usage of KeyTypes

	// getter
	public String getMasterType() {
		return masterType;
	}

	// getter
	public String getType() {
		return type;
	}

	// setter
	public void setType(String type) {
		this.type = type;
	}
	
	/* This method is used to establish key order
	 * an abstract toString method is declared
	 * this method must be implemented by any class that extends Collectable
	 * toString is used to establish the string representation of an object
	*/  
	public abstract String toString();

	/* This method is used to compare toString of objects
	 * the compareTo method is implemented from the Comparable interface
	 * it compares the string representations of two Collectable objects 
	 * using their toString methods
	*/
	public int compareTo(Collectable obj) {
		return this.toString().compareTo(obj.toString());
	}

	// static print method used by extended classes
	public static void print(Collectable[] objs) {
		// print 'Object' properties

		System.out.println(objs.getClass() + " " + objs.length);

		// print 'Collectable' properties
		if (objs.length > 0) {

			Collectable obj = objs[0];	// Look at properties of 1st element
			System.out.println("Collectable: " + obj.getType() + " listed by " + obj.getSortKey());
		}

		// print "Collectable: Objects'
		for(Object o : objs)	// observe that type is Opaque
			System.out.println(o);

		System.out.println();
	}
}

In [39]:
/*
 * Pet class extends Collectable and defines abstract methods
 */
public class Pet extends Collectable {
	// Class data
	public enum KeyType implements KeyTypes {title, name, age, color}
	public static KeyTypes key = KeyType.title;  // static initializer
	public static KeyTypes sortKey = KeyType.title;
	public static void setOrder(KeyTypes key) { Pet.key = key; }
	public static void sort(Collectable[] c, KeyType key) { 
		Pet.key = key; 
		Pet.sortKey = key; 
		Arrays.sort(c); 
		Pet.key = KeyType.title; 
	}

	// Instance data
	private final String name;
	private final int age;
	private final String color;

	/* constructor
	 *
	 */
	public Pet(String name, int age, String color)
	{
		super.setType("Pet");
		this.name = name;
		this.age = age;
		this.color = color;
	}

	/* 'Collectable' requires getKey to help enforce KeyTypes usage */
	@Override
	protected KeyTypes getKey() { return Pet.key; }
	protected KeyTypes getSortKey() { return Pet.sortKey; }

	/* Getters / Accessors
	 * 
	 */
	public String getName() { return this.name; }
	public int getAge() { return this.age; }
	public String getColor() { return this.color; }

	
	/* 'Collectable' requires toString override
	 * toString provides data based off of Static Key setting
	 */
	@Override
	public String toString()
	{
		String output="";
		if (KeyType.name.equals(this.getKey())) {
			output += this.name;
		} else if (KeyType.age.equals(this.getKey())) {
			output += "00" + this.age;
			output = output.substring(output.length() - 2);
		} else if (KeyType.color.equals(this.getKey())) {
			output += this.color;
		} else {
			output += super.getType() + ": " + this.name + ", " + this.color + ", " + this.age;
		}
		return output;
		
	}

	// Test data initializer
	public static Pet[] pets() {
		return new Pet[]{
				new Pet("Lion", 8, "Gold"),
				new Pet("Pig", 3, "Pink"),
				new Pet("Robin", 7, "Red"),
				new Pet("Cat", 10, "Black"),
				new Pet("Kitty", 1, "Calico"),
				new Pet("Dog", 14, "Brown")
		};
	}
	
	/* main to test Pet class
	 * 
	 */
	public static void main(String[] args)
	{
		// print with title
		Pet.setOrder(KeyType.title);
		Collectable[] pets = Pet.pets();

		for (Pet.KeyType key : Pet.KeyType.values()) {
			Pet.sort(pets, key);
			Collectable.print(pets);
		}
	}

}
Pet.main(null);

class [LREPL.$JShell$14J$Pet; 6
Collectable: Pet listed by title
Pet: Cat, Black, 10
Pet: Dog, Brown, 14
Pet: Kitty, Calico, 1
Pet: Lion, Gold, 8
Pet: Pig, Pink, 3
Pet: Robin, Red, 7

class [LREPL.$JShell$14J$Pet; 6
Collectable: Pet listed by name
Pet: Cat, Black, 10
Pet: Dog, Brown, 14
Pet: Kitty, Calico, 1
Pet: Lion, Gold, 8
Pet: Pig, Pink, 3
Pet: Robin, Red, 7

class [LREPL.$JShell$14J$Pet; 6
Collectable: Pet listed by age
Pet: Kitty, Calico, 1
Pet: Pig, Pink, 3
Pet: Robin, Red, 7
Pet: Lion, Gold, 8
Pet: Cat, Black, 10
Pet: Dog, Brown, 14

class [LREPL.$JShell$14J$Pet; 6
Collectable: Pet listed by color
Pet: Cat, Black, 10
Pet: Dog, Brown, 14
Pet: Kitty, Calico, 1
Pet: Lion, Gold, 8
Pet: Pig, Pink, 3
Pet: Robin, Red, 7



In [40]:
/*
 * Car class extends Collectable and defines abstract methods
 */
public class Car extends Collectable {
	// Class data
	public enum KeyType implements KeyTypes {title, make, model, year, color}
	public static KeyTypes key = KeyType.title;  // static initializer
	public static KeyTypes sortKey = KeyType.title;
	public static void setOrder(KeyTypes key) { Car.key = key; }
	public static void sort(Collectable[] c, KeyType k) { 
		Car.key = key; 
		Car.sortKey = key; 
		Arrays.sort(c); 
		Car.key = KeyType.title; 
	}

	// Instance data
    private final String make;
    private final String model;
	private final int year;
	private final String color;

	/* constructor
	 *
	 */
	public Car(String make, String model, int year, String color)
	{
		super.setType("Car");
		this.make = make;
        this.model = model;
		this.year = year;
		this.color = color;
	}

	/* 'Collectable' requires getKey to help enforce KeyTypes usage */
	@Override
	protected KeyTypes getKey() { return Car.key; }
	protected KeyTypes getSortKey() { return Car.sortKey; }

	/* Getters / Accessors
	 * 
	 */
	public String getMake() { return this.make; }
    public String getModel() { return this.model; }
	public int getYear() { return this.year; }
	public String getColor() { return this.color; }

	
	/* 'Collectable' requires toString override
	 * toString provides data based off of Static Key setting
	 */
	@Override
	public String toString()
	{
		String output="";
		if (KeyType.make.equals(this.getKey())) {
			output += this.make;
        } else if (KeyType.model.equals(this.getKey())) {
			output += this.model;
		} else if (KeyType.year.equals(this.getKey())) {
			output += this.year;
		} else if (KeyType.color.equals(this.getKey())) {
			output += this.color;
		} else {
			output += super.getType() + ": " + this.make + ", " +  this.model + ", " + this.color + ", " + this.year;
		}
		return output;
		
	}

	// Test data initializer
	public static Car[] cars() {
		return new Car[]{
				new Car("Ford", "Fusion", 2015, "Guard"),
				new Car("Ford", "Excursion", 2003, "Green"),
                new Car("Ford", "F-350", 1997, "Green"),
                new Car("Cadillac", "Boughman", 1969, "Black"),
                new Car("Acura", "RL", 2006, "Silver")
		};
	}
	
	/* main to test Car class
	 * 
	 */
	public static void main(String[] args)
	{
		// print with title
		Car.setOrder(KeyType.title);
		Collectable[] cars = Car.cars();

		for (Car.KeyType key : Car.KeyType.values()) {
			Car.sort(cars, key);
			Collectable.print(cars);
		}
	}

}
Car.main(null);

class [LREPL.$JShell$28F$Car; 5
Collectable: Car listed by title
Car: Acura, RL, Silver, 2006
Car: Cadillac, Boughman, Black, 1969
Car: Ford, Excursion, Green, 2003
Car: Ford, F-350, Green, 1997
Car: Ford, Fusion, Guard, 2015

class [LREPL.$JShell$28F$Car; 5
Collectable: Car listed by title
Car: Acura, RL, Silver, 2006
Car: Cadillac, Boughman, Black, 1969
Car: Ford, Excursion, Green, 2003
Car: Ford, F-350, Green, 1997
Car: Ford, Fusion, Guard, 2015

class [LREPL.$JShell$28F$Car; 5
Collectable: Car listed by title
Car: Acura, RL, Silver, 2006
Car: Cadillac, Boughman, Black, 1969
Car: Ford, Excursion, Green, 2003
Car: Ford, F-350, Green, 1997
Car: Ford, Fusion, Guard, 2015

class [LREPL.$JShell$28F$Car; 5
Collectable: Car listed by title
Car: Acura, RL, Silver, 2006
Car: Cadillac, Boughman, Black, 1969
Car: Ford, Excursion, Green, 2003
Car: Ford, F-350, Green, 1997
Car: Ford, Fusion, Guard, 2015

class [LREPL.$JShell$28F$Car; 5
Collectable: Car listed by title
Car: Acura, RL, Silver, 200

In [52]:
import java.util.ArrayList;
import java.util.List;

public class Collections {
	public static void main(String[] args)
	{
		List<Collectable> collection = new ArrayList<>();
		collection.addAll(List.of(Pet.pets()));
		collection.addAll(List.of(Car.cars()));

		for (Collectable c : collection) {
			System.out.println(c);
		}

    }
}
Collections.main(null);


Pet: Lion, Gold, 8
Pet: Pig, Pink, 3
Pet: Robin, Red, 7
Pet: Cat, Black, 10
Pet: Kitty, Calico, 1
Pet: Dog, Brown, 14
Car: Ford, Fusion, Guard, 2015
Car: Ford, Excursion, Green, 2003
Car: Ford, F-350, Green, 1997
Car: Cadillac, Boughman, Black, 1969
Car: Acura, RL, Silver, 2006


## Hacks
Build your own Collectable objects and sort them.